In [2]:
import pandas as pd

In [4]:
data = pd.read_csv('us_customer_data 2.csv')
data2= pd.read_csv('transaction_data 1.csv')



In [6]:
import re

def standardize_phone(number, default_country="1"):
    if number is None or pd.isna(number):
        return None
    cleaned = str(number)
    cleaned = cleaned.split("x")[0]

    # Keep only digits and '+' from the main number
    cleaned = re.sub(r'[^0-9+]', '', cleaned)

    # If it starts with '00', replace with '+'
    if cleaned.startswith("00"):

        cleaned = "+" + str(int(cleaned[0:3]))+ "-"+ cleaned[3:]

    # If it doesn’t start with '+', assume default country code
    if not cleaned.startswith("+"):
        cleaned = "+" + default_country + "-"+ cleaned

    return cleaned


# Apply the standardization function to the phone column
data['phone'] = [standardize_phone(num) for num in data['phone']]


In [7]:
data.head(5)

,customer_id,name,email,phone,address,registration_date,loyalty_status
0,1,Michelle Kidd,vayala@example.net,None,"USNS Santiago, FPO AE 80872",2025-01-25,Gold
1,2,Brad Newton,taylorcatherine@example.net,+17595188536,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver
2,3,Larry Torres,dsanchez@example.net,+13235253094,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze
3,4,Kimberly Price,jessicaknight@example.com,+19476334224,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold
4,5,Matthew Phillips,qwilliams@example.com,+18696505682,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold


In [8]:
# Cleaining the Name column
data['name'] = data['name'].str.strip().str.title()


In [9]:
data.head(5)

,customer_id,name,email,phone,address,registration_date,loyalty_status
0,1,Michelle Kidd,vayala@example.net,None,"USNS Santiago, FPO AE 80872",2025-01-25,Gold
1,2,Brad Newton,taylorcatherine@example.net,+17595188536,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver
2,3,Larry Torres,dsanchez@example.net,+13235253094,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze
3,4,Kimberly Price,jessicaknight@example.com,+19476334224,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold
4,5,Matthew Phillips,qwilliams@example.com,+18696505682,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold


In [10]:
# Generating Email Id where it is missing
def generate_email(name,email, domain="example.com"):
    if email is not None and not pd.isna(email):
        return email.lower()
    if name is None or pd.isna(name):
        return None
    name_parts = name.split()
    if len(name_parts) < 2:
        return name.lower() + "@" + domain
    first_name = name_parts[0].lower()
    last_name = name_parts[-1].lower()
    return f"{first_name}.{last_name}@{domain}"

data['email'] = [generate_email(name,email) for name, email in zip(data['name'], data['email'])]


In [11]:
result = data[['customer_id', 'name', 'email', 'phone']]

In [17]:
data2.head(5)

,transaction_id,customer_id,amount,transaction_date,product_category,payment_method,store_location
0,1,565,2992.47,2025-03-10 01:20:54,Sports,Debit Card,New York
1,2,323,2041.87,2025-01-02 15:24:19,Clothing,Cash,New York
2,3,398,107.35,2025-02-16 03:49:01,Beauty,Debit Card,Online
4,5,547,3063.28,2025-06-14 04:28:53,Clothing,PayPal,Los Angeles
5,6,752,441.49,2025-03-28 06:15:55,Electronics,Cash,New York


In [16]:
data2 = data2.dropna(subset=['amount'])

In [21]:
data2=data2.groupby('customer_id')['amount'].max()

In [24]:
df = pd.merge(result, data2, on='customer_id', how='inner').sort_values(by='amount', ascending=False)

In [28]:
df = df[df['amount'] > 1000]
df.head(10)

,customer_id,name,email,phone,amount
547,866,Michael Ross,woodsangela@example.com,+17923165191,4998.17
538,852,Andrew Garcia,andrew.garcia@example.com,+19389825413,4995.63
509,806,Savannah White,xmcfarland@example.net,+17283292155,4988.06
71,102,Larry Hill,garrettrebecca@example.org,+14517023092,4979.11
204,314,Craig Winters,kaufmanangela@example.com,+18846867841,4978.25
231,358,Joe Chen,spencerjames@example.net,+18066257149,4977.87
289,445,Mallory Mann,charles11@example.net,+18698124023,4974.03
309,476,Valerie Jackson,patriciabates@example.org,+19632388643,4948.70
25,33,Erin Stephens,kyle83@example.com,+19476262105,4934.62
587,924,Jacqueline Long,danielle16@example.org,+13163723789,4929.15
